<a href="https://colab.research.google.com/github/irullah/prosaindata/blob/main/crawling_berita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests_html
!pip install BeautifulSoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 12.9 MB/s eta 0:00:00
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=fef0942a5fe12ad657b52845ccb88d60b844fcfbf9cac8865e857d5f1b1be728
  Stored in directory: /root/.cache/pip/wheels/e5/33/1f/68392720485b3ecf125a69e700baaab7624616deedea2fa6e2
Successfully built parse
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from requests_html import HTMLSession 
from bs4 import BeautifulSoup as bs
import pandas as pd

s = HTMLSession()

In [ ]:
def getData(url):
  r = s.get(url)
  soup = bs(r.text,"html.parser")
  return soup

def getNextPage(soup):
  link = soup.find(["a"], {"class" : "paging__link paging__link--next"})
  if not link:
    return
  else:
    url = link["href"]
    return url

In [ ]:
url = "https://news.kompas.com/search"

In [ ]:
Go = True
hasil = {"judul": [], "isi" : []}
jumlah_link = 0
while Go and jumlah_link < 50:
  soup_indeks = getData(url)
  medias = soup_indeks.find_all(["div"], class_ = "article__list clearfix")
  for media in medias:
    link = media.find(["a"], class_ = "article__link")
    jumlah_link += 1
    if jumlah_link > 50:
      break
    else:
      link_berita = link["href"]
      soup_berita = getData(link_berita)
      judul = soup_berita.find(["h1"], class_ = "read__title")
      hasil["judul"].append(judul.string)
      div_berita = soup_berita.find_all(["div"], class_ = "read__content")
      for berita in div_berita:
        isi = berita.find_all(["p"])
        y = [y.text for y in isi]
        content = ''.join(y).replace("KOMPAS.com -","")
        hasil["isi"].append(content)
  soup = getData(url)
  url = getNextPage(soup)
  if not url:
    Go = False

In [ ]:
df = pd.DataFrame(list(zip(hasil["judul"],hasil["isi"])),columns = ["Judul","Isi"])

In [ ]:
df.head()

,Judul,Isi
0,Jadwal Kereta Pertama dari Serpong ke Tanah Ab...,Kereta Commuter Line paling pagi dari Serpong...
1,Jadwal Kereta Pertama dari Duri ke Manggarai ...,Kereta Commuter Line paling pagi dari Duri ke...
2,"Soal Masuk Pukul 05.00, KPAI Minta Pemprov NTT...","JAKARTA, Komisi Perlindungan Anak Indonesia (..."
3,Pakar Hukum: KUHP Baru Bisa Digugat ke MK Sete...,"JAKARTA, Pakar hukum Universitas Udayana, I D..."
4,"PAN: Tsamara Amany Kelihatan Mulai ""Enjoy"" den...","JAKARTA, Partai Amanat Nasional (PAN) membebe..."


In [ ]:
df.tail()

,Judul,Isi
45,Ditanya soal Dugaan Suap dan Gratifikasi Rafae...,"JAKARTA, Deputi Pencegahan dan Monitoring Kom..."
46,"Bantah Nikah Siri dengan Linda, Teddy Minahasa...","JAKARTA, Terdakwa kasus peredaran narkotika j..."
47,BERITA FOTO: KPK Sebut Rafael Alun Punya ''Gen...,"JAKARTA, Komisi Pemberantasan Korupsi (KPK) m..."
48,Polda Metro Janji Jerat Mario Dandy dengan Pas...,"JAKARTA, Polda Metro Jaya bakal menjerat Mari..."
49,"Kelola 2.500 Lebih Kekayaan Intelektual, BRIN ...","JAKARTA, Badan Riset dan Inovasi Nasional (BR..."


In [ ]:
df.to_csv('kompas_last50.csv')